<a href="https://colab.research.google.com/github/meronoumer/learning-agentic-ai-/blob/main/WebSurfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU tavily-python langchain-google-genai langchain-tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.9 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch
from langchain.agents import create_agent

In [3]:
# Step 0: Configure your API keys
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [5]:
# Step 1: Create the LLM
llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest")

# Step 2: Create the search tool
search_tool = TavilySearch(
    max_results=5,
    search_depth="basic",
    include_answer=True,
    include_raw_content=False
)

In [6]:
# Step 3: Create the agent with a system prompt and the tool
agent_prompt = """You are a helpful research assistant.

When answering questions about current events, recent news, or anything that
might have changed after your training data, ALWAYS use the web search tool
first. Do not rely on your training data for:
- Award shows, elections, or recent events
- Current prices, statistics, or rankings
- News from the past year
- Anything the user indicates is "recent" or "latest"

Search first, then answer based on what you find."""

agent = create_agent(
    model=llm,
    tools=[search_tool],
    system_prompt=agent_prompt
)

In [7]:
# Step 4: Use the agent
response = agent.invoke({
    "messages": [("user", "Who won the most important awards at the Grammys this year, 2026?")]
})

print(response["messages"])

[HumanMessage(content='Who won the most important awards at the Grammys this year, 2026?', additional_kwargs={}, response_metadata={}, id='6001aa25-6c01-4593-a906-9ba0ce33b87b'), AIMessage(content="I can search for the Grammy winners for the most recent ceremony, but the year 2026 has not happened yet, so there will be no results for that year.\n\nWhich year's Grammy Awards are you interested in? For example, the **2024 Grammy Awards** covered music released in 2023.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite-preview-09-2025', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c69a1-acc2-7022-ba73-67b63bf997dd-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1610, 'output_tokens': 70, 'total_tokens': 1680, 'input_token_details': {'cache_read': 0}})]


In [10]:
# Step 4a: Print the response using chunking
for chunk in agent.stream(
    {"messages": [{"role": "human", "content": "Who won the most important awards at the Grammys this year, 2026?"}]},
    stream_mode="updates"
):
  for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'id': '1fdf366b-a6cc-4744-a5e8-95ec7ab8bf44', 'name': 'tavily_search', 'args': {'time_range': 'year', 'query': 'Grammy Awards 2026 winners most important awards'}}]
step: tools
content: [{'type': 'text', 'text': '{"query": "Grammy Awards 2026 winners most important awards", "follow_up_questions": null, "answer": "Kendrick Lamar and Bad Bunny were the top winners at the 2026 Grammy Awards, with Bad Bunny making history as the first Latin artist to win Album of the Year. Lady Gaga also had a successful night, earning multiple awards including Best Dance Pop Recording.", "images": [], "results": [{"url": "https://www.bbc.com/news/articles/czx1z0qwy77o", "title": "Grammy Awards 2026: All the winners and nominees - BBC", "content": "# All the winners at the 2026 Grammy Awards. **The 68th Grammy Awards have taken place in Los Angeles, with Kendrick Lamar the night\'s biggest winner for the second year in a row.**. * **Winner: Kendrick Lamar feat SZ